# 5章 クラスと継承 項目38-40
2023/3/15

担当：須藤

#### 必要なモジュールの `import`

In [2]:
# Reproduce book environment
import random
random.seed(1234)

import logging
from pprint import pprint
from sys import stdout as STDOUT

# Write all output to a temporary directory
import atexit
import gc
import io
import os
import tempfile

TEST_DIR = tempfile.TemporaryDirectory()
atexit.register(TEST_DIR.cleanup)

# Make sure Windows processes exit cleanly
OLD_CWD = os.getcwd()
atexit.register(lambda: os.chdir(OLD_CWD))
os.chdir(TEST_DIR.name)

def close_open_files():
    everything = gc.get_objects()
    for obj in everything:
        if isinstance(obj, io.IOBase):
            obj.close()

atexit.register(close_open_files)

<function __main__.close_open_files()>

## 項目38 単純なインタフェースにはクラスの代わりに関数を使う

### フックと defaultdict クラス

Python には、関数を渡すことによって振る舞いをカスタマイズできる組み込み API が多くある。このような仕組みをフックと呼び、API は、実行中にそのコードをコールバックします。例えば、`list` 型の `sort` メソッドは、オプションとして `key` 引数を取り、各要素のソート値を決定するのに使います（`key` 引数の詳細は「項目14 `key` 引数を使い複雑な基準でソートする」参照）。`key` フックとして組み込み関数 `len` を指定し、名前のリストを長さによってソートするコードを次に示します。

In [3]:
# Example 1
names = ['Socrates', 'Archimedes', 'Plato', 'Aristotle']
names.sort(key=len)
print(names)

['Plato', 'Socrates', 'Aristotle', 'Archimedes']


他の言語だと、フックが抽象クラスで定義される。Python では、フックの多くは、きちんと定義された引数と戻り値を持ち、状態のない関数です。関数は記述が容易で、クラスよりも定義が単純な ので、フックには理想的です。関数がフックとして機能するのは、Python が関数をファーストクラスとしているからです。すなわち、Python 言語では、関数とメソッドが他の値と同様に渡され参照できるからです。
例えば、`defaultdict` クラス（背景については「項目17 内部状態の欠損要素を扱うには `setdefault` ではなく `defaultdict` を使う」参照）の振る舞いをカスタマイズしたいとします。このデータ 構造では、キーが見つからなかったら、そのたびに呼ばれる引数なしの関数を指定することができます。その関数は、見つからなかったキーが辞書で持っているべきデフォルト値を返さねばなりません。キーが見つからないとログを取り、デフォルト値として0を返すフックを次のように定義します。

例えば、`defaultdict` クラス（背景については「項目17 内部状態の欠損要素を扱うには `setdefault` ではなく `defaultdict` を使う」参照）の振る舞いをカスタマイズしたいとする。

`defaultdict`は、[Pythonのドキュメント](https://docs.python.org/3/library/collections.html)で以下のように説明されている

- `class collections.defaultdict(default_factory=None, /[, ...])`

`defaultdict`は`default_factory`という引数を持ち、これが`None`であるとき`KeyError`例外を発生させる。

In [63]:
from collections import defaultdict

dd_error = defaultdict()
dd_error["key1"].append(1)

KeyError: 'key1'

`default_factory`には引数を持たない関数を与えることが出来、その関数に応じて新たな値の格納の仕様を指定できる。

例えば、`defaultdict` `dd` に新たなキーを追加するとき、リストに格納して要素を与えたいとすれば次のように`list`を渡せばよい。

In [74]:
dd_list = defaultdict(list)
dd_list["key1"] += [1]
dd_list["key2"] += [2,"two"]
print(dd_list)
print(dict(dd_list))

defaultdict(<class 'list'>, {'key1': [1], 'key2': [2, 'two']})
{'key1': [1], 'key2': [2, 'two']}


結果を見ると分かるように、`defaultdict`そのものを`print`すると、初期値を指定する関数と辞書と二つの情報を`defaultdict`型として表示される。

辞書のみを表示したい場合は`dict()`でラップすればよい。

`defaultdict`の引数`default_factory`に自身で作った以下の性質を満たす関数を指定することで仕様をカスタマイズできる：
- 引数を取らない
- 初期値を`return`として返す
 
例えば次のような関数を指定することができる。

In [77]:
# Example 2
def log_missing():
    print('Key added')
    return 3

dd_lm = defaultdict(log_missing)
dd_lm["key1"] += 1
dd_lm["key2"] += 2
print(dict(dd_lm))

Key added
Key added
{'key1': 4, 'key2': 5}


ここの挙動としては、
1. `defaultdict` `dd_lm`に新たなキー`key1`を追加しようとする
2. `dd_lm`にキー`key1`は存在しないため、初期化のため`log_missing()`を実行する
3. `log_missing()`を実行するため、`print('Key added')`が実行され、デフォルト値として`3`が与えられる
4. `3 + 1 = 4`が`key1`の値として格納される

以下のサンプルコード Example 3 でも同様の挙動を確認することができる。

In [5]:
# Example 3
from collections import defaultdict

current = {'green': 12, 'blue': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9),
]
result = defaultdict(log_missing, current)
print('Before:', dict(result))
for key, amount in increments:
    result[key] += amount
print('After: ', dict(result))

Before: {'green': 12, 'blue': 3}
Key added
Key added
After:  {'green': 12, 'blue': 20, 'red': 5, 'orange': 9}


このように`defaultdict`は、引数に`log_missing` のような関数を与えることで、新しいキーが追加される度にログを表示するといった機能を付与することができ、API の構築とテストが容易になる。

### defaultdict と `__call__`

次に`defaultdict` に渡すデフォルト値のフックで、見つからないキーの総数を数えることを考える。一つ目の方法として、状態を持つクロージャを使うことが考えられる（詳細は「項目21 クロージャが変数スコープとどう関わるかを把握しておく」参照）。このようなクロージャをデフォルト値のフックとして用いるヘルパー関数を次のように定義する。

In [6]:
# Example 4
def increment_with_report(current, increments):
    added_count = 0

    def missing():
        nonlocal added_count  # Stateful closure
        added_count += 1
        return 0

    result = defaultdict(missing, current)
    for key, amount in increments:
        result[key] += amount

    return result, added_count

この関数を実行すると、`defaultdict` 自体は `missing` というフックが状態（カウント数）を保持していることを知らずとも、期待された結果である`2`が得られる。

In [7]:
current = {'green': 12, 'blue': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9),
]

# Example 5
result, count = increment_with_report(current, increments)
assert count == 2
print(result)

defaultdict(<function increment_with_report.<locals>.missing at 0x0000027B8C4A7010>, {'green': 12, 'blue': 20, 'red': 5, 'orange': 9})


このような状態を保持する関数をクロージャとして実装しているだけなので、機能を自由に追加できるメリットがある。

このように「状態をクロージャをフックとして実装する」際の問題点は、状態を持たない関数の例に比べてかなり読みにくい点である。

この点を解消する方法として、次のように、追跡したい状態をカプセル化した軽量なクラスを定義する方法が考えられる。

In [8]:
# Example 6
class CountMissing:
    def __init__(self):
        self.added = 0

    def missing(self):
        self.added += 1
        return 0

他言語の場合、`defaultdict` を修正して、`CountMissing` を受け入れられるインタフェースにする必要があることが多い。一方、Python では、関数がファーストクラス（変数として代入等が可能）なため、オブジェクトで直接 `CountMissing.missing` メソッドを参照して、それをデフォルト値フックとして `defaultdict` に渡すことができる。

In [9]:
current = {'green': 12, 'blue': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9),
]

# Example 7
counter = CountMissing()
result = defaultdict(counter.missing, current)  # Method ref
for key, amount in increments:
    result[key] += amount
assert counter.added == 2
print(result)

defaultdict(<bound method CountMissing.missing of <__main__.CountMissing object at 0x0000027B8D7EFD90>>, {'green': 12, 'blue': 20, 'red': 5, 'orange': 9})


このようなヘルパークラスを使って、状態を持つクロージャの振る舞いを提供することは、先ほどの `increment_with_report` 関数よりもコードが明確になります。しかし、`CountMissing` クラス 単独で見ると、このクラスの目的が何であるかがすぐにはわかりません。誰が、`CountMissing` オブジェクトを作るのでしょうか。 誰が `missing` メソッドを呼ぶのでしょうか。 クラスには、将来他のパブリックメソッドが必要となるのでしょうか。`defaultdict` でどのように使われるのかを確かめるまでは、このクラスは謎のままです。

このような状況を切り抜けるために、Python はクラスで特殊メソッド `__call__` を定義できます。`__call__` では、関数のようにオブジェクトを呼び出すことができます。これは、そのようなインスタ ンスに対して、組み込み関数 `callable`が `True` を返すようにします。このように実行できるオブジェクトはすべて、「呼び出し可能」と呼ばれます。


In [10]:
# Example 8
class BetterCountMissing:
    def __init__(self):
        self.added = 0

    def __call__(self):
        self.added += 1
        return 0

counter = BetterCountMissing()
assert counter() == 0
assert callable(counter)

次のように、`BetterCountMissing` インスタンスを `defaultdict` のデフォルト値フックとして使って、追加された中で、キーがなかったものの個数を追跡します。

In [11]:
# Example 9
counter = BetterCountMissing()
result = defaultdict(counter, current)  # Relies on __call__
for key, amount in increments:
    result[key] += amount
assert counter.added == 2
print(result)

defaultdict(<__main__.BetterCountMissing object at 0x0000027B8C523910>, {'green': 12, 'blue': 20, 'red': 5, 'orange': 9})


これは、`CountMissing.missing` の例よりもずっとわかりやすくなっています。`__call__` メソッドは、クラスのインスタンスがどこかで（API フックのように）関数引数として使われてもよいことを示唆します。これは、新たにコードを読んだ人に、クラスの基本的な振る舞いに対する責任のありかを示します。クラスの目的が状態を持つクロージャとして働くことであるという強い手がかりを与えます。
何よりも良いことは、`__call__` を使っても、何が起こっているかについて `defaultdict` が何も 知らなくて良いことです。`defaultdict` に必要なことは、デフォルト値をフックする関数だけです。Python は、何を行いたいかに応じて、単純な関数インタフェースを満たすさまざまな方法をたくさん用意しています。


### 覚えておくこと

- Python のコンポーネント間の単純なインタフェースは、クラスを定義してインスタンス化しなくても、たいてい関数で済ませられる。
- Python では関数とメソッドの参照はファーストクラスなので、他の型同様、式中で使うことができる。
- 特殊メソッド `__call__` を使用すると、クラスのインスタンスを Python の普通の関数として呼び出すことが可能になる。
- 状態を保守するために関数が必要な場合、状態を持つクロージャを定義する代わりに、`__call__` メソッドを提供するクラスを定義することを考える。

## 項目39 @classmethod ポリモルフィズムを使ってオブジェクトをジェネリックに構築する

Python では、オブジェクトだけでなくクラスもポリモルフィズムをサポートします。それは、どういう意味で、どんな利点があるのでしょうか。

ポリモルフィズムを使用すると、階層を成す複数のクラスでそれぞれ独自のバージョンのメソッドを実装できるようになります。この方式では、多くのクラスが同じインタフェース、あるいは、抽象基底クラスを実現しながら、異なった機能を提供します（例えば、「項目43 カスタムコンテナ型は `collections.abc` を継承する」参照）。

例えば、`MapReduce` の実装を書いていて、入力データを表す共通クラスが欲しいとします。サブクラスで定義する必要のある `read` メソッドを持つクラスを次のように定義します。

In [12]:
# Example 1
class InputData:
    def read(self):
        raise NotImplementedError

データをディスクのファイルから読み込む、`InputData` の具象サブクラスもあります。

In [13]:
# Example 2
class PathInputData(InputData):
    def __init__(self, path):
        super().__init__()
        self.path = path

    def read(self):
        with open(self.path) as f:
            return f.read()

`PathInputData` のような `InputData` のサブクラスをいくつでも作ることができ、それぞれが処理するデータを返すための標準的な読み込みインタフェースを実装することができます。他の `InputData` のサブクラスでは、ネットワークから読み込んだり、データを透過的に解凍したりすることなどができます。

入力データを標準的に消費する `MapReduce` の `Worker` にも同様の抽象インタフェースが欲しくなったとしましょう。


In [14]:
# Example 3
class Worker:
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None

    def map(self):
        raise NotImplementedError

    def reduce(self, other):
        raise NotImplementedError

具体的な `Worker` サブクラスとして、単純な改行のカウンタを適用したい `MapReduce` 関数として定義します。

In [15]:
# Example 4
class LineCountWorker(Worker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count('\n')

    def reduce(self, other):
        self.result += other.result

この実装は取り組み甲斐がありますが、一番大きな難関でもあります。これらの部品すべてを連結するのは何でしょうか。妥当なインタフェースと抽象化を備えた良いクラスの集合があっても、オブジェクトが作られて初めて役に立つものです。オブジェクトを構築して、`MapReduce` を統合する責任は誰が負うのでしょうか。

最も単純な方式は、ヘルパー関数を使って、オブジェクトを構築して連携する作業を自分の手で行うことです。ディレクトリの内容をリストして、そこに含まれる各ファイルに対する `PathInputData` インスタンスを作ります。


In [16]:
# Example 5
import os

def generate_inputs(data_dir):
    for name in os.listdir(data_dir):
        yield PathInputData(os.path.join(data_dir, name))

次に、`generate_inputs` で返された `InputData` インスタンスを作ります。

In [17]:
# Example 6
def create_workers(input_list):
    workers = []
    for input_data in input_list:
        workers.append(LineCountWorker(input_data))
    return workers

複数のスレッドに実行ステップを `map` することによって、これらの `Worker` インスタンスを並列に実行します（「項目53 スレッドはブロッキングI/O に使い、並列性に使うのは避ける」）。そして、`reduce` を繰り返し呼び出して、結果を1つの最終的な値にまとめます。

In [18]:
# Example 7
from threading import Thread

def execute(workers):
    threads = [Thread(target=w.map) for w in workers]
    for thread in threads: thread.start()
    for thread in threads: thread.join()

    first, *rest = workers
    for worker in rest:
        first.reduce(worker)
    return first.result

最後に、これらの部品をまとめて、各ステップを実行する関数にします。

In [19]:
# Example 8
def mapreduce(data_dir):
    inputs = generate_inputs(data_dir)
    workers = create_workers(inputs)
    return execute(workers)

テスト用の入力ファイルにこの関数を実行した結果は素晴らしいものでした。

In [20]:
# Example 9
import os
import random

def write_test_files(tmpdir):
    os.makedirs(tmpdir)
    for i in range(100):
        with open(os.path.join(tmpdir, str(i)), 'w') as f:
            f.write('\n' * random.randint(0, 100))

tmpdir = 'test_inputs'
write_test_files(tmpdir)

result = mapreduce(tmpdir)
print(f'There are {result} lines')

There are 4360 lines


何が問題でしょうか。大きな問題は、この `mapreduce` 関数がまったく（ジェネリックプログラミングの意味での）ジェネリックではないことです。別の `InputData` や `Worker` といったサブクラスを書いたなら、`generate_inputs` や `create_workers` を書き直して、 `mapreduce` 関数でも対応しなければいけません。

この問題を突き詰めると、オブジェクトを構築するジェネリックな方式が必要だということになります。他の言語では、コンストラクタポリモルフィズムを使って、各 `InputData` サブクラスが専用のコンストラクタを提供し、`MapReduce` を統合するヘルパーメソッドからジェネリックに利用すれば解決できます。しかし、`Python` では、`__init__` という単一のコンストラクタメソッドしか使えません。すべての `InputData` サブクラスが、同じコンストラクタのみ使うようにするのは現実的ではありません。

この問題を解く最良の方法は、クラスメソッドポリモルフィズムを使うものです。これは、 `InputData.read` で用いたインスタンスメソッドポリモルフィズムと本質的に同じですが、構築されたオブジェクトにではなく、クラス全体について適用される点が異なります。

この方式を `MapReduce` クラスに適用しましょう。`InputData` クラスを拡張して、共通のイ ンタフェースを用いる、新たな `InputData` インスタンスを作る責任を負う、ジェネリックな `@classmethod` を追加します。


In [21]:
# Example 10
class GenericInputData:
    def read(self):
        raise NotImplementedError

    @classmethod
    def generate_inputs(cls, config):
        raise NotImplementedError

`generate_inputs` は、`GenricInputData` の具象サブクラスが解釈する必要がある設定パラメータの辞書を取ります。次のように、`config` を使って入力ファイルを探すディレクトリを指定します。

In [22]:
# Example 11
class PathInputData(GenericInputData):
    def __init__(self, path):
        super().__init__()
        self.path = path

    def read(self):
        with open(self.path) as f:
            return f.read()

    @classmethod
    def generate_inputs(cls, config):
        data_dir = config['data_dir']
        for name in os.listdir(data_dir):
            yield cls(os.path.join(data_dir, name))

同様にして、`create_workers` ヘルパー関数を `GenericWorker` クラスの一部として作ることができます。パラメータ `input_class` に `GenericInputData` のサブクラスを渡して、必要な入力を生成することにします。`GenericWorker` の具象サブクラスのインスタンスを、`cls()` をジェネリックなコンストラクタとして呼び出し、作成します。

In [23]:
# Example 12
class GenericWorker:
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None

    def map(self):
        raise NotImplementedError

    def reduce(self, other):
        raise NotImplementedError

    @classmethod
    def create_workers(cls, input_class, config):
        workers = []
        for input_data in input_class.generate_inputs(config):
            workers.append(cls(input_data))
        return workers

上の `input_class.generate_inputs` という呼び出しが、示したいクラスポリモルフィズムであることに注意してください。`create_workers` が `cls` を呼び出すという方式が、`__init__` メソッドを直接使って `GenericWorker` オブジェクトを構築する方式に替わるものであることもわかるでしょう。

`GenericWorker` の具象サブクラスへの影響は、スーパークラスを変更することだけです。

In [24]:
# Example 13
class LineCountWorker(GenericWorker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count('\n')

    def reduce(self, other):
        self.result += other.result

最後に、`mapreduce` 関数を書き直して、`create_workers` を呼び出して完全にジェネリックにします。

In [25]:
# Example 14
def mapreduce(worker_class, input_class, config):
    workers = worker_class.create_workers(input_class, config)
    return execute(workers)

新しい `Worker` を試験用のファイルに実行すると、前の実装のときと同じ結果が生成されます。ただし、`mapreduce` 関数がジェネリックになったことで、より多くの引数が必要であることが異なります。

In [26]:
# Example 15
config = {'data_dir': tmpdir}
result = mapreduce(LineCountWorker, PathInputData, config)
print(f'There are {result} lines')

There are 4360 lines


今度は、`GenericInputData` や `GenericWorker` サブクラスを他に好きなように書くことができます。関係するコードを書き直す必要はありません。

### 覚えておくこと

- Python は、クラスに対して、`__init__` メソッドという1つのコンストラクタしかサポートしていない。
- クラスに対して、代わりのコンストラクタを定義するために `@classmethod` を使う。
- 具象サブクラスを作成して連携するジェネリックな方式を提供するには、クラスメソッドポリモルフィズムを使う。

## 項目 40 super を使ってスーパークラスを初期化する

サブクラスからスーパークラスを初期化する古いやり方は、スーパークラスの `__init__` メソッドをサブクラスのインスタンスで直接呼び出すことでした。

In [27]:
# Example 1
class MyBaseClass:
    def __init__(self, value):
        self.value = value

class MyChildClass(MyBaseClass):
    def __init__(self):
        MyBaseClass.__init__(self, 5)

    def times_two(self):
        return self.value * 2

foo = MyChildClass()
assert foo.times_two() == 10

この方式は、単純な階層では問題ありませんが、多くの場合にうまくいきません。

クラスが、多重継承（一般には避けるべきことです。「項目41 `Mix-in` クラスで機能合成を考える」参照）によって影響を受けているとき、スーパークラスの `__init__` メソッドを直接呼び出すと、予期せぬ振る舞いに遭遇することがあります。

問題は、`__init__` メソッドの呼び出し順序がすべてのサブクラス間で規定されてはいないことです。例えば、インスタンスの `value` フィールドを操作する2つのスーパークラスを次のように定義したとします。

In [28]:
# Example 2
class TimesTwo:
    def __init__(self):
        self.value *= 2

class PlusFive:
    def __init__(self):
        self.value += 5

スーパークラスを次に示すような順序で定義します。

In [29]:
# Example 3
class OneWay(MyBaseClass, TimesTwo, PlusFive):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

このオブジェクトを構築すると、 スーパークラスの順序に合致するような結果を出します。

In [30]:
# Example 4
foo = OneWay(5)
print('First ordering value is (5 * 2) + 5 =', foo.value)

First ordering value is (5 * 2) + 5 = 15


次に、同じスーパークラスで、 ただし順序が異なるクラスを定義します（`PlusFive` の後に
`TimesTwo` が来る)。

In [31]:
# Example 5
class AnotherWay(MyBaseClass, PlusFive, TimesTwo):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

しかし、スーパークラスのコンストラクタへの呼び出しの `TimesTwo.__init__` と `PlusFive.__init__` を `Oneway` クラスと同じ順序にしたので、このクラスの振る舞いは、その定義でのスーパークラスの順序に対応しません。 継承基底クラスと `__init__` 呼び出しとの食い違いというこの問題は、同定が難しく、コードを初めて見る人には、特に理解が難しいでしょう。

In [32]:
# Example 6
bar = AnotherWay(5)
print('Second ordering value is', bar.value)

Second ordering value is 15


別の問題がダイヤモンド継承で生じます。ダイヤモンド継承とは、2つの異なるサブクラスから継承したサブクラスがあり、かつその2つのクラスが継承階層で同じスーパークラスを持っている場合に生じます。ダイヤモンド継承では、共通のスーパークラスの `__init__` メソッドが何回も実行され、予期せぬ振る舞いを引き起こします。例えば、`MyBaseClass` を継承する2つのサブクラスを次のように定義します。

In [33]:
# Example 7
class TimesSeven(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value *= 7

class PlusNine(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value += 9

そして、これら2つのクラスを継承するサブクラスを定義して、`MyBaseClass` をダイヤモンドの頂点にします。

In [34]:
# Example 8
class ThisWay(TimesSeven, PlusNine):
    def __init__(self, value):
        TimesSeven.__init__(self, value)
        PlusNine.__init__(self, value)

foo = ThisWay(5)
print('Should be (5 * 7) + 9 = 44 but is', foo.value)

Should be (5 * 7) + 9 = 44 but is 14


2番目のスーパークラスのコンストラクタ `PlusNine.__init__` の呼び出しでは、`MyBaseClass.___init__` が2回目に呼び出されたところで、`self.value` が5にリセットされるのです。その結果、`TimesSeven.__init__` コンストラクタの効果がまったく無視され、`self.value` の計算は5 + 9 = 14となります。この振る舞いは驚くべきもので、より複雑な場合にはデバッグが困難です。

この問題を解決するために、Python は組み込み関数 `super` と標準メソッド解決順序（Method Resolution Order : MRO）を用意しました。`super` は、ダイヤモンド階層の共通スーパークラスが一度しか実行されないことを保証します（別の例は「項目48 サブクラスを`__init_subclass__` で検証する」参照）。MRO は、C3 線形化と呼ばれるアルゴリズムに従ってスーパークラスの初期化順を定義しました。

もう一度、次のようにダイヤモンド型の階層を作りますが、今度は `super` を使ってスーパークラスを初期化します。

In [35]:
# Example 9
class MyBaseClass:
    def __init__(self, value):
        self.value = value

class TimesSevenCorrect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value *= 7

class PlusNineCorrect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value += 9

ダイヤモンドの頂点のコンストラクタ `MyBaseClass.__init__` は、今度は1回しか実行されません。他のスーパークラスは、`class` 文で規定された順序で実行されます。

In [36]:
# Example 10
class GoodWay(TimesSevenCorrect, PlusNineCorrect):
    def __init__(self, value):
        super().__init__(value)

foo = GoodWay(5)
print('Should be 7 * (5 + 9) = 98 and is', foo.value)

Should be 7 * (5 + 9) = 98 and is 98


この順序は、最初は逆に見えるかもしれません。`TimesSevenCorrect.__init_` _が最初に実行 されるべきじゃなかったのか？結果は、(57) +9 = 44のはずではなかったか？答えは「いいえ」 です。この順序が、MRO がこのクラスで定義された順番と一致しています。MRO 順序は、`mro` と呼ばれるクラスメソッドで得られます。

In [37]:
# Example 11
mro_str = '\n'.join(repr(cls) for cls in GoodWay.mro())
print(mro_str)

<class '__main__.GoodWay'>
<class '__main__.TimesSevenCorrect'>
<class '__main__.PlusNineCorrect'>
<class '__main__.MyBaseClass'>
<class 'object'>


`GoodWay(5)` を呼び出すと、`TimesSevenCorrect.__init__` が呼ばれ、それが `PlusNineCorrect.__init__` を呼び出し、それが `MyBaseClass.__init__` を呼び出します。ダイヤモンドの頂点に達すると、初期化メソッドのすべては、その `__init__` 関数が呼ばれたのと逆順で作業をします。`MyBaseClass.__init__` は、`value` に5を代入します。`PlusNineCorrect.__init__` は9を足して、`value` を14にします。`TimesSevenCorrect.__init__` が7を掛けて `value` を98にします。`super().__init__` の呼び出しは、多重継承を頑健にするだけでなく、`MyBaseClass.__init__` をサブクラスの中から呼び出すよりも保守性が大幅に良くなります。後で、`MyBaseClass` の名前を変えたり、`TimesSevenCorrect` と `PlusNineCorrect` の `__init__` メソッドを変更しなくても他のスーパークラスから継承するようにできます。

関数 `super` にはオプションの引数が2つあります。アクセスしようとする MRO でのスーパークラスから見たクラスの型と、そのインスタンスです。これらの引数をコンストラクタで使うと次のようになります。

In [38]:
# Example 12
class ExplicitTrisect(MyBaseClass):
    def __init__(self, value):
        super(ExplicitTrisect, self).__init__(value)
        self.value /= 3
assert ExplicitTrisect(9).value == 3

しかし、これらの引数はインスタンスの初期化に必須ではありません。Python コンパイラは自動的に正しい引数（`__class__` と `self`）をクラス定義中に `super` が引数なしで呼ばれた場合に補います。これは、次の3つが等価なことを意味します。

In [39]:
# Example 13
class AutomaticTrisect(MyBaseClass):
    def __init__(self, value):
        super(__class__, self).__init__(value)
        self.value /= 3

class ImplicitTrisect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value /= 3

assert ExplicitTrisect(9).value == 3
assert AutomaticTrisect(9).value == 3
assert ImplicitTrisect(9).value == 3

`super` に引数を渡さないといけないのは、スーパークラスの実装の特定の機能にサブクラスからアクセスしなければいけない場合だけです（例：機能をラップしたり再利用する場合）。

### 覚えておくこと

- Python の標準メソッド解決順序 (MRO) は、スーパークラスの初期化順序とダイヤモンド継承の問題を解消する。
- スーパークラスを初期化するには、組み込み関数 `super` を引数なしで使う。
